<a id="back_to_top">

# Advanced SQL Cheat Sheet

**NOTE:** The examples will be based on PostgreSQL dialect and will be using Python's `with` [context management](https://jerrynsh.com/when-to-use-context-managers-in-python/) to ensure we close our database connection

#### Table of Contents

- [row_number() over (PARTITION BY ORDER BY) as new_column](#over_partion_by_order_by)
- [Obtaining the first n records in each partition](#first_n)
- [Using `lag()` function](#lag)
- [Using a "bare" or "empty" `over()` with `min()`, `max()`, etc](#bare_over)

In [1]:
import duckdb
import pandas as pd
pd.options.display.max_rows=1000
pd.options.display.max_columns=100

In [4]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT
            timestamp
            , plant_location
            , value
        FROM
            power_usage
        """
    ).df()

df

,timestamp,plant_location,value
0,2022-02-01 09:00:00,Plant B,6.2
1,2022-02-01 10:00:00,Plant B,7.3
2,2022-02-01 11:00:00,Plant B,8.1
3,2022-02-01 12:00:00,Plant B,9.2
4,2022-02-01 13:00:00,Plant B,10.4
5,2022-02-01 14:00:00,Plant B,11.9
6,2022-02-01 15:00:00,Plant B,12.3
7,2022-02-01 16:00:00,Plant B,10.8
8,2022-02-01 17:00:00,Plant B,9.1
9,2022-02-01 18:00:00,Plant B,7.5


<a id="over_partion_by_order_by">

## row_number() over (PARTITION BY ORDER BY) as new_column

[[back to top]](#back_to_top)

In [5]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT
            timestamp
            , plant_location
            , value
            , row_number() over (
                PARTITION BY
                    plant_location
                ORDER BY
                    timestamp
            ) as row_num
        FROM
            power_usage"""
    ).df()
    
df

,timestamp,plant_location,value,row_num
0,2022-03-01 00:00:00,Plant A,14.2,1
1,2022-03-01 01:00:00,Plant A,12.7,2
2,2022-03-01 02:00:00,Plant A,10.9,3
3,2022-03-01 03:00:00,Plant A,8.9,4
4,2022-03-01 04:00:00,Plant A,7.3,5
5,2022-03-01 05:00:00,Plant A,6.8,6
6,2022-03-01 06:00:00,Plant A,7.9,7
7,2022-03-01 07:00:00,Plant A,9.4,8
8,2022-03-01 08:00:00,Plant A,11.1,9
9,2022-03-01 09:00:00,Plant A,12.7,10


**Use Case:** Let's say you want to know the first earliest value by each plant location, then with the above, we can add a filter in the where clause with the outer select:

In [6]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT *
        FROM (
        SELECT
            timestamp
            , plant_location
            , value
            , row_number() over (
                PARTITION BY
                    plant_location
                ORDER BY
                    timestamp
            ) as row_num
        FROM
            power_usage
        ) as inner_select
        WHERE
            row_num = 1
        """
    ).df()
    
df

,timestamp,plant_location,value,row_num
0,2022-03-01 00:00:00,Plant A,14.2,1
1,2022-02-01 09:00:00,Plant B,6.2,1


or use `SELECT DISTINCT ON`:

In [7]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT DISTINCT ON (plant_location)
            timestamp,
            plant_location,
            value,
            row_number() over (
                PARTITION BY
                    plant_location
                ORDER BY
                    timestamp
            ) as row_num
        FROM
            power_usage
        ORDER BY
            plant_location, timestamp
        """
    ).df()
    
df

,timestamp,plant_location,value,row_num
0,2022-03-01 00:00:00,Plant A,14.2,1
1,2022-02-01 09:00:00,Plant B,6.2,1


<a id="first_n">

## Obtaining the first n records in each partition

[[back to top]](#back_to_top)

Obtaining the first 3 earliest values in each plant location:

In [8]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT *
        FROM (
        SELECT
            timestamp
            , plant_location
            , value
            , row_number() over (
                PARTITION BY
                    plant_location
                ORDER BY
                    timestamp
            ) as row_num
        FROM
            power_usage
        ) as inner_select
        WHERE
            row_num <= 3
        """
    ).df()
    
df

,timestamp,plant_location,value,row_num
0,2022-03-01 00:00:00,Plant A,14.2,1
1,2022-03-01 01:00:00,Plant A,12.7,2
2,2022-03-01 02:00:00,Plant A,10.9,3
3,2022-02-01 09:00:00,Plant B,6.2,1
4,2022-02-01 10:00:00,Plant B,7.3,2
5,2022-02-01 11:00:00,Plant B,8.1,3


If you want to use rank, look into using `rank()` or `dense_rank()` depending on how you want to handle ties.  Most often than not, you will probably want to use `dense_rank()` as it does not skip ranks when a tie occurs.

<a id="lag">

## Using `lag()` function

[[back to top]](#back_to_top)

Let's say we have yearly employee salary data

In [9]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT
            employee_id, 
            salary_year,
            salary
        FROM
            employee_salary
        """
    ).df()
    
df

,employee_id,salary_year,salary
0,1001,2015,50000
1,1002,2015,60000
2,1001,2016,55000
3,1002,2016,62000
4,1001,2017,57000
5,1002,2017,64000
6,1001,2018,59000
7,1002,2018,66000
8,1001,2019,61000
9,1002,2019,68000


`lag()` function is useful if you want to obtain the previous value. Syntax: `lag( [column_nam] , [offset], default if n/a )`

In [10]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT
            employee_id, 
            salary_year,
            salary,
            lag(salary, 1, 0) OVER (PARTITION BY employee_id ORDER BY salary_year) as one_yr_lag
        FROM
            employee_salary
        """
    ).df()
    
df

,employee_id,salary_year,salary,one_yr_lag
0,1001,2015,50000,0
1,1001,2016,55000,50000
2,1001,2017,57000,55000
3,1001,2018,59000,57000
4,1001,2019,61000,59000
5,1001,2020,63000,61000
6,1001,2021,65000,63000
7,1001,2022,67000,65000
8,1002,2015,60000,0
9,1002,2016,62000,60000


By having `one_yr_lag` column available, we can now calculate the difference in salary compared to previous 1 year.

In [11]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT
            employee_id
            , salary_year
            , salary
            , CASE
                WHEN one_yr_lag != 0 THEN salary - one_yr_lag
                ELSE 0 END AS one_yr_salary_diff
        FROM (
        SELECT
            employee_id, 
            salary_year,
            salary,
            lag(salary, 1, 0) OVER (PARTITION BY employee_id ORDER BY salary_year) as one_yr_lag
        FROM
            employee_salary
        ) as inner_select
        """
    ).df()
    
df

,employee_id,salary_year,salary,one_yr_salary_diff
0,1001,2015,50000,0
1,1001,2016,55000,5000
2,1001,2017,57000,2000
3,1001,2018,59000,2000
4,1001,2019,61000,2000
5,1001,2020,63000,2000
6,1001,2021,65000,2000
7,1001,2022,67000,2000
8,1002,2015,60000,0
9,1002,2016,62000,2000


<a id="bare_over">

## Using a "bare" or "empty" `over()` with `min()`, `max()`, etc

[[back to top]](#back_to_top)

When people think of using a `min()` or `max()` function, they may immediately think of having to use a `GROUP BY` also.  But using the GROUP BY causes you to lose or limit the columns you want to see at the same time.  This is where using a "bare" `over()` idiom or pattern is handy.  Basically, this allows you to keep your original data or table and just tack on your min or max column.

In [12]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT
            employee_id, 
            salary_year,
            salary,
            min(salary) OVER (PARTITION BY employee_id) as min_salary
        FROM
            employee_salary
        """
    ).df()
    
df

,employee_id,salary_year,salary,min_salary
0,1001,2015,50000,50000
1,1001,2016,55000,50000
2,1001,2017,57000,50000
3,1001,2018,59000,50000
4,1001,2019,61000,50000
5,1001,2020,63000,50000
6,1001,2021,65000,50000
7,1001,2022,67000,50000
8,1002,2015,60000,60000
9,1002,2016,62000,60000


versus having to use `GROUP BY`:

In [13]:
with duckdb.connect('./data/misc.duckdb') as con:
    df = con.sql(
        """
        SELECT
            employee_id
            , min(salary) as min_salary
        FROM
            employee_salary
        GROUP BY
            employee_id
        """
    ).df()
    
df

,employee_id,min_salary
0,1001,50000
1,1002,60000
